In [ ]:
import numpy as np
from tqdm import tqdm
import pickle


In [ ]:
# Follow instruction SEFT repository to get access to P12 data
# https://github.com/BorgwardtLab/Set_Functions_for_Time_Series/tree/master#available-datasets

In [ ]:
import tensorflow_datasets as tfds
import medical_ts_datasets

In [ ]:
import tensorflow as tf

In [ ]:
# load it one time for : train dev test

splitname = 'train' # train, dev ,test
p12, p12_info = tfds.load(name='physionet2012', split='train', as_supervised=True, with_info=True)

In [ ]:


data = []
diag_times = []
for i,d in tqdm(enumerate(p12)):
    x,y=d
    static, timestamps,diagnosis,mask, l = x
    data_dict ={}
    data_dict['static'] = static.numpy()
    data_dict['timestamps'] = timestamps.numpy()
    data_dict['diagnosis'] = diagnosis.numpy()
    data_dict['masks'] = mask.numpy()
    data_dict['target'] = y.numpy()
    data_dict['length'] = l.numpy()
    data.append(data_dict)
    #print(diagnosis.numpy().shape)
    
filename = f'p12_{splitname}.pickle'

with open(filename, 'wb') as fp:
    pickle.dump(data, fp)


with open(filename, 'rb') as fp:
    data_p12 = pickle.load(fp)
    
len(data_p12)

In [ ]:
7671+1917+2400+12

# Preparing stat files

In [ ]:
diag_total = np.zeros(37)
count_total = np.zeros(37)
count_total_p = np.zeros(37)
patient_avg_ = np.zeros(37)
min_total = np.array([np.inf]*37)
max_total = np.array([-np.inf]*37)
val_dict = {f'idx{i}': [] for i in range(37)}

for d in tqdm(data):
    diag = d['diagnosis']
    mask = d['masks']
    timestamp = d['timestamps']
    #print(timestamp)
    total_hr = timestamp[-1] - timestamp[0]
    #print(diag.shape)#, diag)#, np.nansum(diag, axis=0))
    diag_total += np.nansum(diag, axis=0)
    count_total += np.sum(mask, axis=0)
    for i,(v,m) in enumerate(zip(diag.T, mask.T)):
        val_dict[f'idx{i}']+=list(v[m])
        #print(i,'?',v[m])
        if len(v[m]) >0:
            maxof = max(max_total[i],max(v[m]))
            minof = min(min_total[i],min(v[m]))
            max_total[i] = maxof
            min_total[i] = minof
            #print(i, "==>", maxof,minof)
    
    print(np.sum(mask, axis=0)/total_hr)
    patient_avg_ += (np.sum(mask, axis=0)/total_hr)
    #break
    

In [ ]:
patient_avg_/7671 # avg of (avg of diagnosis per variable) per patient

In [ ]:
# sampling rate file

with open('sampling_rate_p12.npy', 'wb') as fp:
    np.save(fp, patient_avg_/7671)
    
with open('sampling_rate_p12.npy', 'rb') as fp:
    sr_p12 = np.load(fp)
    
# https://numpy.org/doc/stable/reference/generated/numpy.save.html

In [ ]:
final_stat = {}
for k in val_dict:
    print(k, min(val_dict[k]), max(val_dict[k]), np.mean(val_dict[k]), np.std(val_dict[k]), np.var(val_dict[k]), len(val_dict[k]))
    final_stat[k] = {"min":min(val_dict[k]),
                    "max": max(val_dict[k]),
                    "mean":np.mean(val_dict[k]),
                    "std": np.std(val_dict[k]),
                    "count": len(val_dict[k]),
                    "var":np.var(val_dict[k]),
                    "median": np.median(val_dict[k], axis=0)}

In [ ]:
filename = 'p12_stat.pickle'
with open(filename, 'wb') as fp:
    pickle.dump(final_stat, fp)
    
with open(filename, 'rb') as fp:
    stat_p12 = pickle.load(fp)
    
len(stat_p12)